<a href="https://colab.research.google.com/github/EyalMK/IntroductionToCloudProgramming-Tutorials/blob/main/Tutorial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Imports
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, HBox, VBox
from google.colab import drive

In [ ]:
### Utilities class
class Utilities:
  def __init__(self):
    pass

  def get_graph_options(self):
    return ['Number of actions by user',
                 'Actions on a document (pie chart)', '']


In [ ]:
### Plot factory class
class PlotFactory:
  def __init__(self):
    self.plot_types = {
            "bar": BarChartPlot,
            "pie": PieChartPlot
        }

  def get_plot(self, plot_type, plot_data, plot_title, plot_x_label, plot_y_label, data_colors=None):
    plot_class = self.plot_types.get(plot_type)
    if plot_class is None:
        raise ValueError("Unsupported plot type")
    return plot_class(plot_data, plot_title, plot_x_label, plot_y_label)

In [ ]:
### Plot class
class Plot:
  def __init__(self, plot_data, plot_title, plot_x_label, plot_y_label, data_colors=None):
    self.plot = None
    self.__plot_data = plot_data
    self.data_colors = self.get_validated_data_colors(data_colors)
    self.plot_title = plot_title
    self.plot_x_label = plot_x_label
    self.plot_y_label = plot_y_label

  def get_plot_data(self):
    return self.__plot_data

  def set_plot_data(self, plot_data=None):
    self.__plot_data = plot_data

  def get_validated_data_colors(self, data_colors):
    if data_colors is None:
      return
    if self.__plot_data is None or self.__plot_data.size != data_colors.length:
      raise Exception("Colors size mismatch")
    return data_colors

  def set_graph_plot(self, ax):
    raise Exception("Template method - inheriting sub-classes must implement this method.")

  def build_plot(self):
    # Template method
    fig, ax = plt.subplots(figsize=(8, 5))
    self.set_graph_plot(ax)
    ax.set_title(self.plot_title)
    ax.set_xlabel(self.plot_x_label)
    ax.set_ylabel(self.plot_y_label)
    self.plot = fig

  def show(self):
    if self.plot is None:
      self.build_plot()
    plt.show()


In [ ]:
### Piechart Plot
class PieChartPlot(Plot):
  def __init__(self, plot_data, plot_title, plot_x_label, plot_y_label, data_colors=None):
    super().__init__(plot_data, plot_title, plot_x_label, plot_y_label, data_colors)

  def set_graph_plot(self, ax):
    ax.pie(self.get_plot_data(), labels=self.get_plot_data().index, colors=self.data_colors, autopct='%1.1f%%', startangle=140)

In [ ]:
### Barchart Plot
class BarChartPlot(Plot):
  def __init__(self, plot_data, plot_title, plot_x_label, plot_y_label, data_colors=None):
    super().__init__(plot_data, plot_title, plot_x_label, plot_y_label, data_colors)

  def set_graph_plot(self, ax):
      ax = plt.bar(self.get_plot_data().index, self.get_plot_data())

In [ ]:
### App class
class App:
  def __init__(self, data_dir=""):
    self.data_frame = None
    self.data_path = data_dir  # Directory in My Drive/* that contains data.json
    self.full_path = ""
    self.utils = Utilities()

  def initialize(self):
    drive_path = "/content/drive"
    self.full_path = f"{drive_path}/MyDrive/{self.data_path}/"  # /content/drive/MyDrive/Cloud-Programming
    drive.mount(drive_path, force_remount=True)
    os.chdir(self.full_path)

  def process_data(self, df: pd.DataFrame) -> pd.DataFrame:
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df.dropna(subset=['Time'], inplace=True)
    df['Description'] = df['Description'].astype('category')
    df['User'] = df['User'].astype('category')
    df['Document'] = df['Document'].astype('category')
    return df

  def setup_widgets(self):
    graph_options = self.utils.get_graph_options()
    self.graph_widget = widgets.Dropdown(
      options = graph_options,
      value = "",
      description='Select Graph:',
      disabled=False,
      layout=widgets.Layout(margin='10px 0px')
    )

    doc_options = list(self.data_frame['Document'].dropna().unique())
    self.document_widget = widgets.Combobox(
        options = doc_options,
        value = doc_options[0] if doc_options else None,
        description='Select Document:',
        placeholder='Search by name',
        ensure_option=True,
        disabled=False,
        layout=widgets.Layout(margin='10px 0px', width='300px')
    )
    self.document_widget.style.description_width = '103px'

    self.error_graph = widgets.HTML(
        value="<div style='color: #ff3333; font-size: 14px; margin-top: 20px;'>Invalid graph chosen</div>",
        layout=widgets.Layout(display='none', margin='10px 0px')
    )

    self.document_widget.observe(self.__update_graph_widget, 'value')
    self.__update_graph_widget()
    vbox_layout = widgets.Layout(width='80%', height='200px')
    ui = VBox([self.document_widget, self.graph_widget, self.error_graph], layout=vbox_layout)
    self.output = widgets.Output(layout=widgets.Layout(margin='20px 0px'))
    self.graph_widget.observe(self.__on_option_chosen, names='value')
    self.document_widget.observe(self.__on_option_chosen, names='value')

    display(ui, self.output)

  def __on_option_chosen(self, change):
    with self.output:
        self.output.clear_output()
        self.error_graph.layout.display = 'none'
        self.__display_graph(self.graph_widget.value, self.document_widget.value)

  def __update_graph_widget(self, *args):
    if self.document_widget.value:
        self.graph_widget.layout.display = 'block'
    else:
        self.graph_widget.layout.display = 'none'

  def __plot_actions_by_user_activity(self):
    user_action_counts = self.data_frame['User'].value_counts()
    pltFactory = PlotFactory()
    plt = pltFactory.get_plot("bar", user_action_counts, "Number of actions by user", "User", "Count")
    plt.show()

  def __plot_actions_on_document(self, document):
    filtered_df = self.data_frame[self.data_frame['Document'] == document]
    action_counts = filtered_df['Description'].value_counts()
    action_counts = action_counts[action_counts > 0]
    pltFactory = PlotFactory()
    plt = pltFactory.get_plot("pie", action_counts, f"Actions on {document}", "", "")
    plt.show()

  def __display_graph(self, graph, document):
    if graph == 'Number of actions by user':
        self.__plot_actions_by_user_activity()
    elif graph == 'Actions on a document (pie chart)':
        self.__plot_actions_on_document(document)
    elif graph == '':
        self.error_graph.layout.display = 'block'
    else:
        self.error_graph.layout.display = 'block'

  def run(self):
    if not os.path.isdir(self.data_path):
      self.initialize()
    self.data_frame = self.process_data(pd.read_json(self.full_path + "data.json"))
    self.setup_widgets()

In [ ]:
### Main
if __name__ == '__main__':
  app = App(data_dir="Cloud-Programming")
  app.initialize()
  app.run()

Mounted at /content/drive
Mounted at /content/drive


Output(layout=Layout(margin='20px 0px'))